In [ ]:
# # Mount
# from google.colab import drive
# drive.mount('/content/gdrive')a

In [ ]:
# Import Library
import numpy as np
import pandas as pd
import datetime as dt

# 전주 동요일 매출과 차이 큰 상위 동 

In [ ]:
gu = pd.read_excel('./gdrive/My Drive/빅콘 대상팀/data/지역데이터/구_동.xlsx')
gs = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/all_amt.csv',parse_dates=['STD_YMD'])

gs = pd.concat([gs.iloc[:,[0,1]],gs.filter(like='GS')],axis=1)
gs = pd.merge(gs,gu,on='HDONG_NM')
gs['CITY'] = gs['HDONG_GU'].apply(lambda x: x[0:2])
gs_seoul = gs.query('CITY == "서울" & STD_YMD > "2020" & HDONG_NM != "상계8동" ' )
gs_seoul= gs_seoul.drop(['HDONG_GU','CITY'],axis=1)

gs_d = pd.DataFrame({'HDONG_NM': gs_seoul.HDONG_NM.unique(),'eat_d':0,'snack_d':0,'drink_d':0})
gs_d = gs_d.set_index('HDONG_NM')

gs_eat = gs_seoul.iloc[:,[0,1,3]].sort_values(['HDONG_NM','STD_YMD'])
gs_snack = gs_seoul.iloc[:,[0,1,4]].sort_values(['HDONG_NM','STD_YMD'])
gs_drink = gs_seoul.iloc[:,[0,1,5]].sort_values(['HDONG_NM','STD_YMD'])

In [ ]:
deli = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/result/배달 원래 값 성장률_2.csv',parse_dates=['일자'])
deli = deli.rename(columns={'일자':'STD_DD'})

## 전주 동요일 매출액 차이 합계

In [ ]:
def gs_diff(data,cat_d,gs_d):
  data['week'] = data.STD_YMD.dt.week
  data['day'] = data.STD_YMD.dt.weekday
  for dong in gs_d.index:
    gs_d.loc[dong][cat_d] = abs(data.loc[data['HDONG_NM']==dong].loc[data['week']== 6].drop(['STD_YMD','HDONG_NM','week'],axis=1).set_index(['day']).loc[5:6] - data.loc[data['HDONG_NM']==dong].loc[data['week']== 5].drop(['STD_YMD','HDONG_NM','week'],axis=1).set_index(['day'])).sum()
    for week_num in range(7,22):
      gs_d.loc[dong][cat_d] = gs_d.loc[dong][cat_d]+abs(data.loc[data['HDONG_NM']==dong].loc[data['week']== week_num].drop(['STD_YMD','HDONG_NM','week'],axis=1).set_index(['day'])  - data.loc[data['HDONG_NM']==dong].loc[data['week']== week_num-1].drop(['STD_YMD','HDONG_NM','week'],axis=1).set_index(['day']) ).sum()

In [ ]:
gs_diff(gs_eat, 'eat_d',gs_d)
gs_diff(gs_snack, 'snack_d',gs_d)
gs_diff(gs_drink, 'drink_d',gs_d)

## 마실거리 매출액 차이

In [ ]:
gs_d.sort_values(by='drink_d',ascending=False).head(5)

,eat_d,snack_d,drink_d
HDONG_NM,,,
명동,373,470,720
소공동,435,617,679
중계2.3동,434,495,637
중계본동,410,563,611
하계2동,432,604,540


In [ ]:
gs_d.sort_values(by='drink_d',ascending=True).head(6)

,eat_d,snack_d,drink_d
HDONG_NM,,,
공릉1동,174,212,207
장충동,223,155,222
월계1동,159,190,229
공릉2동,167,241,266
상계5동,220,216,290
황학동,284,346,304


## 식사 매출액 차이

In [ ]:
gs_d.sort_values(by='eat_d',ascending=False).head(5)

,eat_d,snack_d,drink_d
HDONG_NM,,,
신당동,501,388,502
소공동,435,617,679
중계2.3동,434,495,637
하계2동,432,604,540
중계4동,429,544,511


In [ ]:
gs_d.sort_values(by='eat_d',ascending=True).head(6)

,eat_d,snack_d,drink_d
HDONG_NM,,,
월계1동,159,190,229
공릉2동,167,241,266
공릉1동,174,212,207
월계3동,206,399,394
상계6.7동,216,331,445
상계1동,219,281,340


## 간식 매출액 차이

In [ ]:
gs_d.sort_values(by='snack_d',ascending=False).head(5)

,eat_d,snack_d,drink_d
HDONG_NM,,,
소공동,435,617,679
하계2동,432,604,540
중계본동,410,563,611
중계4동,429,544,511
약수동,331,542,442


In [ ]:
gs_d.sort_values(by='snack_d',ascending=True).head(6)

,eat_d,snack_d,drink_d
HDONG_NM,,,
장충동,223,155,222
월계1동,159,190,229
공릉1동,174,212,207
상계5동,220,216,290
공릉2동,167,241,266
월계2동,235,246,359


# PoC 데이터 생성

## 실제 배달 데이터(2020-05-18 ~ 2020-05-31)

In [ ]:
deli.query( " '2020-05-18'<= STD_DD <= '2020-05-24' ").set_index('STD_DD',drop=True).drop('상계8동',axis=1)

,공릉1동,공릉2동,광희동,다산동,동화동,명동,상계10동,상계1동,상계2동,상계3.4동,상계5동,상계6.7동,상계9동,소공동,신당5동,신당동,약수동,월계1동,월계2동,월계3동,을지로동,장충동,중계1동,중계2.3동,중계4동,중계본동,중림동,청구동,필동,하계1동,하계2동,황학동,회현동
STD_DD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-05-18,4.131054,4.131054,NaN,-6.364360,-6.364360,-11.741742,14.512922,14.512922,14.512922,14.512922,14.512922,14.512922,14.512922,-58.734508,-6.364360,-5.961478,-6.364360,inf,inf,inf,-33.333333,-100.000000,9.090909,9.090909,9.090909,9.090909,NaN,-6.364360,-61.591458,100.000000,100.000000,-100.0,-4.719574
2020-05-19,-40.492476,-40.492476,NaN,-41.886151,-41.886151,4.880451,-51.388889,-51.388889,-51.388889,-51.388889,-51.388889,-51.388889,-51.388889,73.083308,-41.886151,-28.969183,-41.886151,-100.0,-100.0,-100.0,100.000000,inf,-25.000000,-25.000000,-25.000000,-25.000000,NaN,-41.886151,208.931255,0.000000,0.000000,NaN,-5.432358
2020-05-20,-6.896552,-6.896552,NaN,14.035088,14.035088,34.867154,18.214286,18.214286,18.214286,18.214286,18.214286,18.214286,18.214286,37.794829,14.035088,1.232846,14.035088,NaN,NaN,NaN,-25.000000,-40.000000,-44.444444,-44.444444,-44.444444,-44.444444,NaN,14.035088,-23.815169,-50.000000,-50.000000,NaN,-6.385274
2020-05-21,10.370370,10.370370,NaN,9.487179,9.487179,4.261238,-3.776435,-3.776435,-3.776435,-3.776435,-3.776435,-3.776435,-3.776435,9.678542,9.487179,8.154956,9.487179,inf,inf,inf,0.000000,0.000000,80.000000,80.000000,80.000000,80.000000,NaN,9.487179,-21.244164,0.000000,0.000000,NaN,3.516969
2020-05-22,-5.145414,-5.145414,NaN,-0.117096,-0.117096,-9.322095,3.767661,3.767661,3.767661,3.767661,3.767661,3.767661,3.767661,-19.910262,-0.117096,4.225958,-0.117096,0.0,0.0,0.0,-33.333333,33.333333,-33.333333,-33.333333,-33.333333,-33.333333,NaN,-0.117096,33.333333,200.000000,200.000000,NaN,4.956386
2020-05-23,40.094340,40.094340,NaN,16.178195,16.178195,13.204828,32.677761,32.677761,32.677761,32.677761,32.677761,32.677761,32.677761,74.422117,16.178195,1.033959,16.178195,-100.0,-100.0,-100.0,100.000000,-25.000000,50.000000,50.000000,50.000000,50.000000,NaN,16.178195,25.577595,16.666667,16.666667,NaN,9.444431
2020-05-24,21.885522,21.885522,NaN,20.484359,20.484359,-11.163970,22.690992,22.690992,22.690992,22.690992,22.690992,22.690992,22.690992,-7.366030,20.484359,31.972113,20.484359,inf,inf,inf,-25.000000,0.000000,33.333333,33.333333,33.333333,33.333333,NaN,20.484359,-32.220776,-42.857143,-42.857143,NaN,10.310607


In [ ]:
deli.query( " '2020-05-25'<= STD_DD <= '2020-05-31' ").set_index('STD_DD',drop=True).drop('상계8동',axis=1)

,공릉1동,공릉2동,광희동,다산동,동화동,명동,상계10동,상계1동,상계2동,상계3.4동,상계5동,상계6.7동,상계9동,소공동,신당5동,신당동,약수동,월계1동,월계2동,월계3동,을지로동,장충동,중계1동,중계2.3동,중계4동,중계본동,중림동,청구동,필동,하계1동,하계2동,황학동,회현동
STD_DD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-05-25,-1.933702,-1.933702,NaN,-10.134003,-10.134003,-27.324111,-2.323420,-2.323420,-2.323420,-2.323420,-2.323420,-2.323420,-2.323420,-51.747497,-10.134003,-5.757427,-10.134003,0.0,0.0,0.0,-100.000000,-33.333333,-8.333333,-8.333333,-8.333333,-8.333333,NaN,-10.134003,-29.371847,0.0,0.0,NaN,-5.597858
2020-05-26,-32.957746,-32.957746,NaN,-26.467847,-26.467847,39.612227,-45.290200,-45.290200,-45.290200,-45.290200,-45.290200,-45.290200,-45.290200,42.397572,-26.467847,-32.008120,-26.467847,-100.0,-100.0,-100.0,inf,0.000000,-45.454545,-45.454545,-45.454545,-45.454545,NaN,-26.467847,33.653603,25.0,25.0,NaN,-4.955511
2020-05-27,-9.453782,-9.453782,NaN,-2.281369,-2.281369,1.109873,2.956522,2.956522,2.956522,2.956522,2.956522,2.956522,2.956522,98.458200,-2.281369,2.800455,-2.281369,inf,inf,inf,0.000000,50.000000,-16.666667,-16.666667,-16.666667,-16.666667,NaN,-2.281369,-6.654370,0.0,0.0,NaN,-18.757842
2020-05-28,-12.529002,-12.529002,NaN,6.485084,6.485084,-0.325686,1.013514,1.013514,1.013514,1.013514,1.013514,1.013514,1.013514,-26.665468,6.485084,5.565303,6.485084,-100.0,-100.0,-100.0,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,6.485084,-40.462076,60.0,60.0,NaN,23.484517
2020-05-29,18.302387,18.302387,NaN,4.141291,4.141291,3.055461,9.531773,9.531773,9.531773,9.531773,9.531773,9.531773,9.531773,-50.000000,4.141291,-0.893541,4.141291,NaN,NaN,NaN,33.333333,66.666667,60.000000,60.000000,60.000000,60.000000,NaN,4.141291,89.320058,-37.5,-37.5,NaN,-12.119836
2020-05-30,27.354260,27.354260,NaN,17.309942,17.309942,21.935188,40.763359,40.763359,40.763359,40.763359,40.763359,40.763359,40.763359,196.505005,17.309942,20.264347,17.309942,NaN,NaN,NaN,-75.000000,-60.000000,50.000000,50.000000,50.000000,50.000000,NaN,17.309942,-16.923630,60.0,60.0,NaN,9.049487
2020-05-31,25.528169,25.528169,NaN,22.183450,22.183450,-28.305676,12.906725,12.906725,12.906725,12.906725,12.906725,12.906725,12.906725,-42.129020,22.183450,19.682886,22.183450,inf,inf,inf,200.000000,50.000000,-4.166667,-4.166667,-4.166667,-4.166667,NaN,22.183450,35.596797,-62.5,-62.5,inf,10.351156


## 예측 배달 데이터(2020-05-25 ~ 2020-05-31) 

In [ ]:
y_deli = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/result/배달 모든 동 0525_0530예측값.csv')
y_deli.columns=['STD_DD','2020-05-25','2020-05-26','2020-05-27','2020-05-28','2020-05-29','2020-05-30','2020-05-31']
y_deli = y_deli.set_index('STD_DD',drop=True).T.drop('상계8동',axis=1)
y_deli

STD_DD,공릉1동,공릉2동,광희동,다산동,동화동,명동,상계10동,상계1동,상계2동,상계3.4동,상계5동,상계6.7동,상계9동,소공동,신당5동,신당동,약수동,월계1동,월계2동,월계3동,을지로동,장충동,중계1동,중계2.3동,중계4동,중계본동,중림동,청구동,필동,하계1동,하계2동,황학동,회현동
2020-05-25,339.65300,364.12878,-0.181945,214.43774,160.524460,371.32620,85.209400,206.128360,112.764824,130.382190,134.02138,177.431800,115.923120,26.784737,158.889900,150.792540,256.54514,0.684953,0.631311,0.796331,9.732569,7.516799,12.827383,16.794573,9.066509,12.797618,0.181482,213.55376,36.032684,12.460941,9.967044,0.712838,429.80830
2020-05-26,248.49701,255.30893,0.050350,141.71538,107.037346,441.53168,49.420160,115.457924,56.784360,73.664970,71.77202,93.333990,60.363594,24.685677,109.775000,109.704430,167.52301,0.573128,0.541939,0.623693,12.187634,8.159454,9.289070,11.958082,6.617586,9.106613,0.103480,143.51920,37.012615,13.150530,10.448910,0.600871,428.70270
2020-05-27,181.86609,189.20952,0.075241,150.52182,110.751880,521.52704,42.688667,115.390750,55.813602,76.966225,72.60145,96.713036,64.572010,27.022774,109.146050,107.585480,173.00275,0.384883,0.461924,0.439064,11.657815,7.869312,6.898779,8.927649,4.797473,6.431394,0.010818,145.95338,33.119400,14.604703,11.604662,0.510107,425.54987
2020-05-28,190.51710,202.97263,-0.165972,163.35379,118.778760,495.12006,45.343560,120.051820,57.848550,79.085480,77.48573,100.752700,67.108536,31.886404,116.783356,116.231476,186.79416,0.399574,0.438942,0.429228,8.459802,7.374861,7.648798,10.010313,5.208730,7.002424,0.040222,155.12020,34.748020,15.456395,11.980720,0.595680,420.95880
2020-05-29,225.89085,244.53150,0.299524,173.75446,125.577965,590.04420,57.258070,135.781080,68.892090,86.606890,88.47942,116.444150,71.188095,32.796356,128.211360,118.243710,202.77512,0.408099,0.484637,0.482018,9.063678,7.432187,10.170042,12.862610,7.085334,9.673317,-0.023435,164.59052,36.558853,17.995600,14.534624,0.529768,424.62445
2020-05-30,287.67688,314.14883,0.497842,193.79105,140.807340,594.73517,75.018524,167.547870,91.323814,105.714935,110.58611,148.597260,84.913640,31.169300,146.356410,125.315920,231.53930,0.533107,0.624802,0.704105,10.047304,7.764703,13.257112,15.813173,9.051722,12.677993,-0.049516,188.09900,35.268340,14.697618,12.140368,0.528984,428.30148
2020-05-31,332.03088,361.92430,-0.274494,226.13147,166.358640,507.24927,89.025215,207.952120,113.292740,129.787800,136.85858,180.003200,112.252550,35.241573,165.298030,150.649610,265.80502,0.666903,0.729546,0.915519,13.055411,8.947417,14.671907,17.081432,9.801422,14.026681,0.018384,223.14282,37.408405,15.268028,12.762261,0.650251,433.12515


In [ ]:
y_drink = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/result/GRU 모든동 마실거리 0525_0531예측값.csv')
y_drink.columns = ['HDONG_NM','2020-05-25',	'2020-05-26',	'2020-05-27','2020-05-28',	'2020-05-29','2020-05-30','2020-05-31']
y_drink = y_drink.set_index('HDONG_NM',drop=True)

y_eat = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/result/GRU 모든동 식사 0525_0531예측값.csv')
y_eat.columns = ['HDONG_NM','2020-05-25',	'2020-05-26',	'2020-05-27','2020-05-28',	'2020-05-29','2020-05-30','2020-05-31']
y_eat = y_eat.set_index('HDONG_NM',drop=True)

y_snack = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/result/GRU 모든동 간식 0525_0531예측값.csv')
y_snack.columns = ['HDONG_NM','2020-05-25',	'2020-05-26',	'2020-05-27','2020-05-28',	'2020-05-29','2020-05-30','2020-05-31']
y_snack = y_snack.set_index('HDONG_NM',drop=True)

## 카테고리별 PoC 데이터

###마실거리 ALL

In [ ]:
# 실제 마실거리 데이터
real_drink = gs_drink.query(" '2020-05-25' <= STD_YMD <= '2020-05-31' ").set_index(['HDONG_NM','STD_YMD']).unstack().iloc[:,:7]
real_drink

GS_마실거리                        ...                                 
STD_YMD  2020-05-25 2020-05-26 2020-05-27  ... 2020-05-29 2020-05-30 2020-05-31
HDONG_NM                                   ...                                 
공릉1동         26.862     29.440     30.968  ...     32.882     36.456     29.378
공릉2동         27.666     30.171     33.909  ...     35.720     42.525     34.500
광희동          33.440     34.532     36.340  ...     43.263     34.356     24.978
다산동          39.585     39.092     42.952  ...     47.601     41.685     33.998
동화동          30.330     28.269     36.270  ...     35.937     32.453     25.024
명동           53.875     54.944     60.324  ...     61.272     33.120     22.083
상계10동        30.258     30.448     36.801  ...     42.744     46.763     34.776
상계1동         30.515     30.996     34.974  ...     41.830     44.496     42.042
상계2동         48.094     47.736     51.992  ...     61.759     66.882     54.426
상계3.4동       37.125     38.280     40.567  ...     40.424     48.875     43.848
상계5동         30.082     31.850     34.464  ...     36.284     40.456     37.905
상계6.7동       43.618     44.030     45.864  ...     58.201     58.254     47.744
상계9동         38.950     34.047     35.739  ...     46.690     52.884     35.670
소공동          54.796     55.944     64.350  ...     64.070     33.970     25.500
신당5동         32.130     32.860     33.800  ...     38.325     48.314     38.390
신당동          71.410     71.000     74.112  ...     83.514     58.984     44.574
약수동          43.146     39.564     46.440  ...     51.900     58.926     49.491
월계1동         30.240     31.390     33.000  ...     35.712     37.810     32.190
월계2동         34.500     30.616     36.566  ...     42.336     42.800     40.112
월계3동         41.600     43.368     46.842  ...     49.839     55.144     47.880
을지로동         38.352     40.208     42.876  ...     48.076     53.958     35.174
장충동          25.560     24.548     25.560  ...     28.236     26.566     23.424
중계1동         37.440     36.177     42.984  ...     47.790     54.046     49.959
중계2.3동       55.955     57.638     65.010  ...     70.104     68.392     57.130
중계4동         41.612     33.966     45.448  ...     49.764     51.012     46.138
중계본동         45.632     45.339     51.221  ...     53.580     64.561     56.400
중림동          31.815     34.736     38.640  ...     40.368     56.356     55.040
청구동          47.256     43.554     48.117  ...     58.812     61.073     50.596
필동           47.214     47.498     46.585  ...     47.988     31.955     26.270
하계1동         40.553     39.590     45.696  ...     55.151     57.921     38.376
하계2동         47.376     50.876     53.464  ...     59.248     64.260     57.768
황학동          39.349     37.185     39.936  ...     45.356     46.500     41.471
회현동          47.760     47.750     49.724  ...     49.815     34.049     31.570

[33 rows x 7 columns]

In [ ]:
# 마실거리 전주 동요일
before_drink = gs_drink.query(" '2020-05-18' <= STD_YMD <= '2020-05-24' ").set_index(['HDONG_NM','STD_YMD']).unstack().iloc[:,:7]
before_drink

GS_마실거리                        ...                                 
STD_YMD  2020-05-18 2020-05-19 2020-05-20  ... 2020-05-22 2020-05-23 2020-05-24
HDONG_NM                                   ...                                 
공릉1동         25.370     26.280     26.270  ...     30.586     32.238     26.352
공릉2동         25.665     26.158     30.082  ...     33.759     39.402     32.301
광희동          27.720     32.040     36.218  ...     36.828     29.970     23.625
다산동          37.454     38.304     41.776  ...     42.074     47.875     29.610
동화동          30.240     25.718     29.232  ...     33.948     33.525     20.034
명동           49.484     46.886     56.030  ...     56.388     29.520     17.732
상계10동        26.660     26.649     28.801  ...     35.288     37.920     33.784
상계1동         28.638     28.896     31.304  ...     34.848     43.472     32.663
상계2동         44.114     41.895     46.900  ...     54.020     59.130     48.498
상계3.4동       35.970     33.048     37.932  ...     35.392     47.371     38.142
상계5동         29.610     29.610     30.970  ...     33.028     35.136     31.304
상계6.7동       38.690     37.878     44.132  ...     47.988     48.768     43.419
상계9동         33.100     28.574     39.162  ...     40.400     44.576     34.580
소공동          50.995     52.848     61.204  ...     58.460     32.964     21.594
신당5동         29.532     26.350     33.027  ...     39.055     38.046     38.690
신당동          75.719     63.984     69.751  ...     78.596     61.050     38.880
약수동          40.052     40.016     42.976  ...     46.009     48.692     45.825
월계1동         26.394     27.540     30.750  ...     32.178     36.000     29.412
월계2동         28.644     26.316     31.155  ...     35.040     36.358     36.080
월계3동         36.005     32.618     41.472  ...     46.860     48.555     44.067
을지로동         31.208     33.746     37.733  ...     42.074     46.238     24.430
장충동          24.450     19.389     28.638  ...     24.992     27.886     23.010
중계1동         28.600     31.390     36.562  ...     36.900     48.372     38.010
중계2.3동       53.404     49.010     58.830  ...     62.799     65.394     52.244
중계4동         29.670     34.596     40.976  ...     36.406     49.750     34.823
중계본동         39.100     38.532     47.250  ...     47.614     66.612     60.140
중림동          31.512     30.576     33.920  ...     37.950     54.665     49.120
청구동          41.920     44.516     48.544  ...     51.520     56.840     41.658
필동           40.194     39.424     44.460  ...     47.336     32.841     22.644
하계1동         33.900     27.348     44.196  ...     45.539     47.328     38.480
하계2동         47.872     41.676     52.140  ...     50.985     62.075     47.716
황학동          33.744     36.720     39.420  ...     39.050     46.053     36.406
회현동          44.540     42.237     44.045  ...     42.896     33.210     23.919

[33 rows x 7 columns]

In [ ]:
# 마실거리 예측
y_drink

,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31
HDONG_NM,,,,,,,
공릉1동,26.048384,28.363146,30.531990,29.990612,33.148422,35.236633,30.921146
공릉2동,28.214200,29.560230,32.775913,33.423786,36.223648,40.572342,35.872180
광희동,31.737013,34.034912,36.817207,38.507620,39.299297,33.941986,23.865946
다산동,39.370148,41.099770,44.991950,43.501610,45.650284,43.109560,32.825140
동화동,31.155848,28.557545,32.826210,32.342106,36.729300,35.068115,26.067627
명동,51.296700,57.595505,60.317284,60.377373,57.917130,34.170517,21.836155
상계10동,32.292805,28.879120,31.776363,36.755024,40.678490,45.775566,38.814200
상계1동,29.966858,31.691982,34.130047,35.952995,39.858223,44.981834,39.233020
상계2동,48.715280,51.353096,52.951920,52.173347,58.666670,64.202354,55.916573


###식사 ALL

In [ ]:
# 실제 식사 데이터
gs_eat.query(" '2020-05-25' <= STD_YMD <= '2020-05-31' ").set_index(['HDONG_NM','STD_YMD']).unstack().iloc[:,:7]

GS_식사                        ...                                 
STD_YMD  2020-05-25 2020-05-26 2020-05-27  ... 2020-05-29 2020-05-30 2020-05-31
HDONG_NM                                   ...                                 
공릉1동         24.790     25.360     26.228  ...     24.190     22.092     21.756
공릉2동         29.058     27.501     27.768  ...     26.695     28.875     27.692
광희동          27.016     29.779     27.784  ...     28.215     20.580     20.700
다산동          31.605     32.132     33.984  ...     34.071     27.300     21.449
동화동          29.700     28.836     27.714  ...     32.571     23.157     14.912
명동           34.250     33.048     33.528  ...     34.086     15.480     12.801
상계10동        30.504     27.632     26.535  ...     28.808     27.472     23.772
상계1동         25.245     24.272     25.056  ...     23.124     23.793     25.480
상계2동         45.175     42.296     41.674  ...     41.072     38.151     39.762
상계3.4동       49.125     48.952     42.714  ...     45.136     35.625     45.108
상계5동         32.129     34.580     35.136  ...     31.302     35.152     30.495
상계6.7동       32.431     32.844     32.877  ...     36.894     32.266     34.304
상계9동         26.315     28.421     27.324  ...     32.430     26.910     23.142
소공동          39.900     36.704     39.750  ...     35.760     18.170     22.100
신당5동         25.650     27.348     23.900  ...     26.145     25.109     23.760
신당동          53.650     68.000     54.720  ...     46.500     39.274     57.132
약수동          41.595     40.446     38.340  ...     42.900     37.674     37.365
월계1동         28.980     31.648     29.128  ...     31.104     27.455     24.969
월계2동         31.280     31.506     32.618  ...     35.532     30.174     35.970
월계3동         32.032     29.744     28.305  ...     27.972     27.572     29.640
을지로동         22.842     29.232     27.756  ...     28.203     32.430     21.414
장충동          25.134     23.664     24.336  ...     25.350     23.606     18.688
중계1동         27.648     26.040     33.264  ...     36.698     29.646     30.069
중계2.3동       52.700     56.511     52.305  ...     55.752     44.654     41.035
중계4동         32.857     38.046     29.536  ...     33.524     31.005     33.158
중계본동         42.284     39.497     39.562  ...     39.762     43.953     40.185
중림동          37.269     37.544     35.910  ...     37.700     40.588     50.240
청구동          44.484     33.677     40.893  ...     43.836     33.598     38.364
필동           42.312     38.227     38.841  ...     39.345     24.319     21.513
하계1동         33.170     32.635     31.584  ...     32.488     29.412     30.056
하계2동         41.736     55.300     53.953  ...     46.690     45.900     53.592
황학동          33.572     35.631     32.136  ...     33.292     31.992     29.945
회현동          33.360     36.000     34.348  ...     32.964     21.330     24.409

[33 rows x 7 columns]

In [ ]:
# 식사 전주 동요일
gs_eat.query(" '2020-05-18' <= STD_YMD <= '2020-05-24' ").set_index(['HDONG_NM','STD_YMD']).unstack().iloc[:,:7]

GS_식사                        ...                                 
STD_YMD  2020-05-18 2020-05-19 2020-05-20  ... 2020-05-22 2020-05-23 2020-05-24
HDONG_NM                                   ...                                 
공릉1동         25.370     23.433     24.790  ...     25.502     21.060     20.592
공릉2동         28.710     26.732     29.281  ...     28.644     26.235     29.100
광희동          27.810     30.150     29.120  ...     25.389     19.610     20.160
다산동          33.672     34.160     34.720  ...     29.430     31.375     27.730
동화동          23.760     23.870     30.996  ...     28.336     15.975     17.766
명동           33.360     33.677     34.580  ...     30.861     16.488     12.716
상계10동        28.036     28.269     27.639  ...     28.864     26.112     22.960
상계1동         27.434     26.460     27.606  ...     29.280     25.376     23.407
상계2동         36.853     46.550     44.086  ...     42.772     37.084     37.127
상계3.4동       43.230     43.632     49.764  ...     39.424     42.037     39.429
상계5동         34.110     35.820     34.485  ...     31.648     26.976     27.520
상계6.7동       27.030     29.746     33.748  ...     34.185     30.480     33.702
상계9동         29.200     32.942     30.281  ...     30.199     25.872     25.745
소공동          35.960     42.480     37.323  ...     39.664     19.926     20.008
신당5동         25.852     27.455     31.174  ...     27.713     19.278     26.606
신당동          65.772     58.032     59.840  ...     56.056     36.450     30.672
약수동          38.068     36.844     42.296  ...     38.364     38.480     40.185
월계1동         26.477     29.835     27.224  ...     30.411     25.290     27.090
월계2동         30.744     33.626     34.689  ...     31.392     30.478     35.090
월계3동         26.980     29.704     30.132  ...     27.500     26.442     28.083
을지로동         23.406     28.200     26.869  ...     26.160     27.328     19.810
장충동          26.325     26.289     24.050  ...     24.637     21.316     22.620
중계1동         28.864     26.402     29.593  ...     28.800     28.304     29.085
중계2.3동       48.822     52.780     52.947  ...     60.031     49.132     44.104
중계4동         31.046     34.131     29.432  ...     34.226     37.375     28.324
중계본동         38.985     39.444     38.934  ...     40.033     38.532     40.145
중림동          37.336     39.000     37.524  ...     36.685     38.860     51.840
청구동          43.754     36.704     44.992  ...     41.860     36.960     38.645
필동           39.564     41.216     38.493  ...     37.576     27.501     23.732
하계1동         28.900     30.444     34.452  ...     28.137     26.796     27.456
하계2동         44.880     54.662     54.194  ...     56.265     51.570     48.822
황학동          32.190     35.640     32.184  ...     33.770     34.056     30.738
회현동          32.750     34.632     34.270  ...     31.808     23.166     23.718

[33 rows x 7 columns]

In [ ]:
# 식사 예측
y_eat

,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31
HDONG_NM,,,,,,,
공릉1동,24.355309,25.147533,25.373600,24.898783,25.511550,22.969790,21.711922
공릉2동,29.257874,28.046276,28.664474,28.718792,26.929855,28.589067,27.547216
광희동,27.993385,29.509390,29.446205,28.900358,28.751026,20.101755,19.903646
다산동,31.351118,33.174072,33.713450,33.292130,33.778824,26.665220,20.703040
동화동,26.817936,27.502510,27.995403,28.423504,28.617214,23.800549,17.226267
명동,34.110710,34.169647,34.066000,34.046776,33.865715,14.634815,13.083359
상계10동,28.410032,27.554104,29.240486,28.645508,29.278448,28.641857,23.835712
상계1동,24.652107,24.652890,24.652908,24.653020,24.653230,24.653465,24.653172
상계2동,41.570545,42.979820,42.794300,42.532097,42.062504,39.306450,38.930668


### 간식 ALL


In [ ]:
# 실제 간식 데이터
gs_snack.query(" '2020-05-25' <= STD_YMD <= '2020-05-31' ").set_index(['HDONG_NM','STD_YMD']).unstack().iloc[:,:7]

GS_간식                        ...                                 
STD_YMD  2020-05-25 2020-05-26 2020-05-27  ... 2020-05-29 2020-05-30 2020-05-31
HDONG_NM                                   ...                                 
공릉1동         13.764     14.320     13.114  ...     15.498     16.632     13.838
공릉2동         20.271     19.936     17.978  ...     22.610     23.625     21.344
광희동          17.072     16.490     17.296  ...     18.711     17.304     12.696
다산동          22.155     23.664     24.308  ...     26.814     21.735     21.449
동화동          21.330     16.686     20.832  ...     21.582     21.082     17.024
명동           20.000     21.216     24.288  ...     26.220     14.328      9.894
상계10동        15.498     16.632     16.182  ...     20.488     19.695     15.204
상계1동         17.340     16.646     18.705  ...     20.116     21.939     19.600
상계2동         28.078     26.112     26.398  ...     30.955     34.226     29.751
상계3.4동       22.250     21.228     21.357  ...     29.016     27.375     25.830
상계5동         16.198     18.655     19.008  ...     19.646     20.488     19.000
상계6.7동       24.973     26.299     26.442  ...     35.178     40.588     32.640
상계9동         22.610     24.929     24.255  ...     23.460     26.325     19.488
소공동          23.142     26.196     30.000  ...     33.525     16.827     13.056
신당5동         21.870     24.380     24.900  ...     22.680     30.226     26.400
신당동          37.370     36.200     39.360  ...     35.526     32.412     23.736
약수동          41.595     33.264     35.505  ...     43.500     47.978     38.634
월계1동         15.372     14.964     16.808  ...     19.104     20.045     18.705
월계2동         19.412     19.847     16.638  ...     23.112     24.717     24.961
월계3동         22.568     20.488     26.196  ...     24.531     29.280     27.018
을지로동         18.518     16.576     19.548  ...     23.800     29.946     17.200
장충동          12.354     11.424     13.248  ...     17.706     15.836     14.656
중계1동         21.408     21.576     23.220  ...     27.022     30.134     27.612
중계2.3동       32.240     29.946     33.000  ...     41.768     37.350     33.495
중계4동         21.527     18.156     21.736  ...     23.200     22.464     26.196
중계본동         27.156     29.718     31.833  ...     37.083     39.767     33.417
중림동          22.927     18.096     21.000  ...     25.520     35.624     35.360
청구동          28.776     20.825     26.058  ...     39.000     34.540     30.719
필동           22.317     22.479     22.627  ...     26.187     17.679     13.348
하계1동         23.754     21.293     24.528  ...     29.213     27.864     23.192
하계2동         41.454     40.922     45.314  ...     44.919     54.000     49.416
황학동          22.890     22.533     19.760  ...     26.564     29.388     29.041
회현동          24.840     23.000     24.056  ...     24.108     14.931     12.320

[33 rows x 7 columns]

In [ ]:
# 간식 전주 동요일
gs_snack.query(" '2020-05-18' <= STD_YMD <= '2020-05-24' ").set_index(['HDONG_NM','STD_YMD']).unstack().iloc[:,:7]

GS_간식                        ...                                 
STD_YMD  2020-05-18 2020-05-19 2020-05-20  ... 2020-05-22 2020-05-23 2020-05-24
HDONG_NM                                   ...                                 
공릉1동         15.566     13.505     14.504  ...     16.482     16.443     15.264
공릉2동         17.835     19.106     20.025  ...     21.669     21.483     23.280
광희동          14.220     16.200     16.289  ...     16.182     16.280     12.096
다산동          20.008     21.504     24.080  ...     21.582     29.375     21.620
동화동          17.460     21.021     13.944  ...     19.228     20.175     15.498
명동           17.653     20.944     23.140  ...     22.352     14.256      7.612
상계10동        16.082     17.658     17.181  ...     15.136     21.120     17.138
상계1동         15.996     17.640     19.694  ...     19.104     24.856     19.847
상계2동         24.660     27.664     28.006  ...     30.488     30.514     30.003
상계3.4동       20.240     23.328     20.996  ...     25.424     27.178     27.963
상계5동         17.460     17.370     18.810  ...     20.424     22.848     18.748
상계6.7동       22.472     27.713     28.202  ...     30.057     35.560     30.873
상계9동         19.900     21.112     27.392  ...     22.927     29.456     26.600
소공동          22.630     26.496     29.172  ...     28.860     18.778     11.041
신당5동         16.744     19.890     23.871  ...     25.038     30.804     26.182
신당동          32.480     35.898     36.091  ...     38.612     32.700     24.624
약수동          31.868     32.818     35.904  ...     39.615     46.324     41.595
월계1동         15.438     16.575     15.990  ...     17.949     18.900     20.296
월계2동         16.716     19.694     20.460  ...     21.600     23.422     27.390
월계3동         18.620     21.432     25.704  ...     26.290     30.771     29.193
을지로동         13.724     17.108     16.296  ...     22.454     30.988     14.560
장충동          12.975     13.731     13.172  ...     13.987     15.257     12.870
중계1동         21.208     20.382     26.058  ...     26.000     29.928     30.240
중계2.3동       30.178     29.580     34.821  ...     35.292     40.828     36.260
중계4동         15.652     18.321     23.712  ...     28.231     28.875     24.347
중계본동         26.105     24.738     28.854  ...     35.910     39.624     39.525
중림동          19.656     22.464     22.896  ...     24.725     35.670     40.960
청구동          24.235     24.552     29.896  ...     30.940     31.920     33.405
필동           19.404     19.376     21.411  ...     25.376     17.266     12.648
하계1동         18.900     20.124     23.200  ...     25.764     30.508     25.896
하계2동         31.960     45.753     41.870  ...     48.840     59.210     50.876
황학동          23.532     22.788     26.028  ...     25.740     31.992     28.449
회현동          19.519     22.932     21.620  ...     21.392     16.524     11.189

[33 rows x 7 columns]

In [ ]:
# 간식 예측
y_snack

,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31
HDONG_NM,,,,,,,
공릉1동,14.385796,14.347269,14.433614,15.078335,15.421339,15.075735,14.428200
공릉2동,19.538818,19.316708,19.371717,20.526040,22.376642,22.593834,21.485987
광희동,15.296498,17.951267,16.619713,17.092867,18.220726,16.457720,13.849853
다산동,21.329980,24.443533,24.326473,23.562912,23.673576,23.371937,21.223137
동화동,20.468410,19.117304,20.298038,18.944538,19.124174,21.116013,17.077215
명동,19.502796,22.643803,24.262506,24.211748,24.599087,15.891593,9.690062
상계10동,15.210871,15.032204,15.038490,16.058247,17.718720,19.510120,17.375587
상계1동,16.885424,17.197033,17.442600,17.830988,19.414421,22.267382,19.561214
상계2동,26.958752,27.491234,26.819172,27.399060,30.804940,32.247807,29.723618


In [ ]:
## 이후 엑셀로 코딩